In [15]:
import pandas as pd
import numpy as np
import spacy

import sys
sys.path.append('/home/ec2-user/SageMaker/mariano/repositories/tdmstudio-high-recall-information-retrieval-system/')

from utils import tdmstudio

In [3]:
datapath='/home/ec2-user/SageMaker/mariano/datasets/multiculturalism/top2vec/deep_learn_top2vec_topics_only_rel_topics.csv'
df = pd.read_csv(datapath)
df.head()

,Unnamed: 0,topic no.,size,relevant count,example,example(rel),suggestions,relevants,all ids,suggestions (by relevance),...,word_040,word_041,word_042,word_043,word_044,word_045,word_046,word_047,word_048,word_049
0,0,0,400,6,https://proquest.com/docview/1143385056,https://proquest.com/docview/1357031988,1143385056;1125055611;1151428436;1125566253;11...,1357031988;1125706241;1147042690;1151358658;12...,1143385056;1125055611;1151428436;1125566253;11...,1444824582;1151481475;1351093244;1222288596;13...,...,audi,sta,station,shows,licence,hollywood,actress,documentaries,drama,comic
1,1,1,370,6,https://proquest.com/docview/1142439313,https://proquest.com/docview/1371108106,1142439313;1143881816;1148136086;1237203459;12...,1371108106;1143947290;1411791122;1143898495;26...,1142439313;1143881816;1148136086;1237203459;12...,1411867739;1356699776;1282786298;1124990937;11...,...,temporary,appli,arrivals,refugees,skills,provinces,newcomers,applications,grants,wage
2,2,2,237,16,https://proquest.com/docview/1323537660,https://proquest.com/docview/1145627214,1323537660;1148423270;1144117951;1141272360;11...,1145627214;1148596840;1143664837;1143312937;11...,1323537660;1148423270;1144117951;1141272360;11...,1147616449;1434945595;1143806798;1143594520;11...,...,editor,mosaic,preservation,relative,equality,societies,identity,bay,strange,charter
3,3,3,200,4,https://proquest.com/docview/1151120020,https://proquest.com/docview/2122116827,1151120020;1444842268;1316674285;1145780127;14...,2122116827;1141113423;1125527397;1237268511,1151120020;1444842268;1316674285;1145780127;14...,1237671607;1238248945;1356704961;1284751484;11...,...,mother,instruction,knowing,acceptable,learning,children,ask,lunch,find,ita
4,4,4,196,3,https://proquest.com/docview/1143707847,https://proquest.com/docview/1125764078,1143707847;1138432056;1434978441;1400745146;14...,1125764078;1125547053;1125621517,1143707847;1138432056;1434978441;1400745146;14...,1238681375;1412271450;1151314133;1400799649;14...,...,trustees,mcguinty,extending,lic,ruling,charter,elementary,progressive,arguments,christian


In [21]:
ids = [id_ for suggestions_string in df['suggestions'] for id_ in suggestions_string.split(';')]
print(len(ids))
filenames =map(tdmstudio.get_filename , ids)
texts = map(tdmstudio.get_title_and_text,filenames)

suggestions = pd.DataFrame({'id':ids, 'text':texts})
assert suggestions.shape[0]==np.sum(df['size'])-np.sum(df['relevant count'])

suggestions.head()

2710


,id,text
0,1143385056,Taking a swipe at corruption.\n \n ...
1,1125055611,TV viewers in Quebec get option from France.\n...
2,1151428436,Canadian-made TV throws open door to European ...
3,1125566253,The CRTC: not ready for prime time.\n ...
4,1151667436,Europeans to see Canadian TV.\n \n ...


In [129]:
texts = suggestions['text']
texts = [text.lower() for text in texts]
count =0
for text in texts:
    if 'multicultural' in text or 'multiculturalism' in text or 'multicultu' in text:
        count+=1
print(count)
print(f'{(count/len(texts))*100:3.2f} %')

840
31.00 %


# word cloud

In [147]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,3),max_features=1000, use_idf=False)
X = vectorizer.fit_transform(suggestions['text'])
tf = np.sum(X.toarray(), axis=0)
vocab = vectorizer.vocabulary_
tf.shape

vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,3),max_features=1000, use_idf=True)
X = vectorizer.fit_transform(suggestions['text'])
idf = vectorizer.idf_
assert vocab==vectorizer.vocabulary_



In [155]:
tfidf = tf*idf
np.array(vectorizer.get_feature_names())[np.argsort(tfidf)][-100:]

array(['globe', 'white', 'good', 'rights', 'past', 'grants', 'long',
       'italian', 'born', 'know', 'money', 'say', 'population', 'high',
       'need', 'political', 'com', 'heritage', 'day', 'think', 'want',
       'diversity', 'come', 'business', 'old', 'group', 'board',
       'multiculturalism', 'million', 'university', 'parents',
       'vancouver', 'multicultural', 'ottawa', 'immi', 'cbc', 'social',
       'federal', 'students', 'home', 'program', 'montreal', 'minister',
       'policy', 'make', 'groups', 'tion', 'don', 'national', 'way',
       'women', 'museum', 'arts', 'life', 'chinese', 'just', 'family',
       'art', 'ing', 'society', 'time', 'education', '000', 'film',
       'work', 'children', 'ethnic', 'history', 'like', 'ontario',
       'community', 'world', 'immigrant', 'year', 'years', 'language',
       'english', 'cultural', 'public', 'government', 'city', 'country',
       'canadians', 'cent', 'immigration', 'french', 'culture', 'schools',
       'toronto', 'sa

In [152]:
np.argsort(tfidf)

array([360, 487, 853, 389,  74, 117, 539, 804, 328, 909, 616, 324,  78,
       806, 769, 415, 610, 201, 143, 133, 195, 362, 131, 784, 385, 514,
       556, 149, 856, 984, 799, 287,  41, 187,  48,  89, 211, 781,  94,
       584, 962, 641, 619, 384, 748,  13, 447, 640,  57,   8, 276, 878,
        62, 193,  26, 826, 499, 346, 983, 831, 356, 114, 939, 770, 281,
       419, 170,  23, 575, 968, 745, 648, 386,  19, 746, 844, 240, 194,
         6, 103, 241, 898, 801, 135, 423, 294, 767, 141, 722, 207,  96,
       192, 280, 816, 309, 156, 846, 196, 582, 721, 753, 512, 150, 333,
       268, 630, 585, 635, 291, 218, 676, 609, 689, 995, 743, 544, 896,
       839, 712, 106, 866,  93, 958, 216, 265, 671,  66, 441,  86, 857,
       331, 777, 221, 290, 633, 982, 471, 525, 177, 592, 999, 754, 417,
        85, 334, 715, 203, 466, 158, 692, 663, 682, 686, 649, 787, 483,
        12, 615, 655, 581, 238, 478, 410, 841, 924, 343,  47, 534, 765,
       559, 220, 157, 931, 226, 433, 693, 660, 448, 838, 308,  2

In [71]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

ModuleNotFoundError: No module named 'wordcloud'

In [62]:
wordcloud = WordCloud().generate_from_frequencies(data['document_biden_2021'])plt.imshow(wordcloud)

SyntaxError: invalid syntax (1761098772.py, line 1)

In [38]:
nlp = spacy.load("en_core_web_sm")
tokenizer = nlp.tokenizer


tokens = [set(map(lambda x:x.text.lower().strip(), token_list)) for token_list in map(tokenizer, suggestions['text'])]

has_mc = ['multiculturalism' in token_set for token_set in tokens]
print(f'has multiculturalism: {np.sum(has_mc)}')

has_mc = ['multicultural' in token_set for token_set in tokens]
print(f'has multicultural: {np.sum(has_mc)}')


has multiculturalism: 429
has multiculturalism: 589


In [43]:
print(suggestions['text'].iloc[0])

Taking a swipe at corruption.
          
            

 

THEATRE


THEATRE


Taking swipe at corruption


The latest Michel Tremblay work


is more polemic than play


BY RAY CONLOGUE Quebec Arts Correspondent Montreal


MICHEL Tremblay's latest play En circuit ferme stirred up hornets' nest when it received single staged reading last May at the Theatre du Nouveau Monde More polemic than play Tremblay himself calls it pamphlet in the sense of


'Jnoralist's -attack on society it' sat the venality he identifies -in public broadcaster which is quite dearly Radio-Can- ada the Quebec arm of CBC Most of the characters including portly and powerful female' television critic are inspired by real people just suffi ciently camouflaged to ward off libel suits When the show was Revived for Series of 10 public readings this 2all it was anticipated that interest Srould continue to build in' such fashion as to justify a' projected full production next season However the 800 seats at the Monument Na